In [2]:
"""     
Student Name:  ZHANG Xinyue                /        Qiao  Shuyu        /       Li Zhuoxuan
Student ID:    20750194                    /          20747563         /        20740917
Student Email: xzhangfa@connect.ust.hk     /   sqiaoac@connect.ust.hk  /   zlify@connect.ust.hk
Course Name:   MSBD5012
URL in github: https://github.com/orange-neng/MSBD5012-Forest-type-prediction-exploration
"""

'     \nStudent Name:  ZHANG Xinyue                /        Qiao  Shuyu        /       Li Zuoxuan\nStudent ID:    20750194                    /          20747563         /        20740917\nStudent Email: xzhangfa@connect.ust.hk     /   sqiaoac@connect.ust.hk  /   zlify@connect.ust.hk\nCourse Name:   MSBD5012\nURL in github: https://github.com/orange-neng/MSBD5012-Forest-type-prediction-exploration\n'

In [ ]:

'''
2. Classification
After EDA, we will explore several kinds of classification
methods and compare their accuracy.
Different from traditional methods, we apply basic dense
neural network (DNN) (as it's a numerical classification task)
to see whether it can achieve a better performance then other
methods.
'''

# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# import tensorflow as tf
# from sklearn.neural_network import MLPClassifier
# from sklearn.datasets import make_classification
# from sklearn.model_selection import cross_val_score

# Import Datasets
# train = pd.read_csv('./cleaned_train.csv')
# test = pd.read_csv('./cleaned_test.csv')
# raw = pd.read_csv('./covtype.data')
#
# train = raw.iloc[:,1:-1]
# target = raw.iloc[:, -1]
#
# clf = MLPClassifier(random_state=1,
#                     max_iter=300,
#                     hidden_layer_sizes=(30, 20),
#                     activation='logistic')
#
# #3-fold cv
# scores = cross_val_score(clf, train, target,
#                          cv=3, n_jobs=-1, verbose=2)
#
#
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

'''
ID = test.iloc[:,0]
with open("./submission.csv", "w") as subfile:
    subfile.write("Id,Cover_Type\n")
    for i in range(len(result)):
        subfile.write("%s,%s\n"%(ID[i],result[i]))
'''

'''
While using neural network, tuning will always be the hardest problem.
We use the GirdSearchCV provided by sklearn for auto-tuning. 
As batch_size and epochs has a large effect to 
the running speed, for a better running performance, 
let's first check them.
'''

import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam
import pandas as pd

# Function to create model, required for KerasClassifier
def create_model(learning_rate=0.001):
    # create model
    model = Sequential()
    # model.add(Dense(225, input_dim=53, activation='relu'))
    #
    # model.add(Dense(120, activation=activation))
    # model.add(Dense(7, activation='softmax'))
    model.add(Dense(1024, input_dim=54, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 0
np.random.seed(seed)

# load dataset
dataset = np.loadtxt("./covtype_normalize.csv", delimiter=",", skiprows=1)
# get the number of rows and columns
r, c = dataset.shape
np.random.shuffle(dataset)
split = int(0.7*r)
dataset[:, c-1] = dataset[:, c-1] - 1

# split into input (X) and output (Y) variables
x_train = np.asarray(dataset[:split, 1:c-1])
y_train = np.asarray(dataset[:split, c-1])
x_test = np.asarray(dataset[split:, 1:c-1])
y_test = np.asarray(dataset[split:, c-1])


# create model
model = KerasClassifier(build_fn=create_model, verbose=1) #epochs=100, batch_size=500)
# define the grid search parameters
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# dim = [20, 40, 60, 80, 100, 120, 120, 160, 180, 200]
# param_grid = {'learning_rate': [0.001]}
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=1)

# param_rand = {'learning_rate': np.linspace(0.01, 0.1, 10)}
# param_rand = {'learning_rate': [0.001]}
# grid = RandomizedSearchCV(model, param_rand, cv=3, n_jobs=-1)
# grid_result = grid.fit(X, Y)

# summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

history = model.fit(x_train, y_train, epochs=100, batch_size = 2048, validation_data = (x_test, y_test), use_multiprocessing=True)
# print('training acc.:', history.history['acc'][-1],'\n','test acc.:', (history.history['val_acc'])[-1])
pd.DataFrame(history.history).to_csv("loghis.csv")
